#### About this file
1. create xlsx file in test data format __'context', 'true_response_agent'__
2. keep tracking conversations
3. evaluate model using Trainer method by following format: __prompt = context + "agent:" , pred_response = model.generate(prompt)__

In [1]:
#Create test data format
import json
import gzip
import numpy as np
import pandas as pd

In [31]:

filepath = "data/abcd_v1.1.json.gz"

with gzip.open(filepath, 'rt', encoding='utf-8') as f:
    data = json.load(f)
    
# Create empty dataframe variable df
df = pd.DataFrame(columns=["convo_id","context", "true_agent_response"])

# Seperate convo
convo_id = 1 

# There are three keys ['dev','train','test']
# We use test column and convert them into test_data format
for element in data["test"]:
    
    # column1
    context = ""
    # column2 
    # keep appending agent's response till each conversation ends
    true_agent_response = ""
    
    # conversation btwn customer and agent stored in original column
    for conversation in element["original"]:
        #to distinguish speaker (agent,customer) and convo (text)
        speaker, text = conversation
        
        # delete action: ....
        if speaker == 'action' or 'Action' in context:
                continue   
        
        # Build the context based on speaker
        context = f"{context} {speaker}: {text}"
    
        # if text is the convo between existing customer and agent 
        if speaker == "agent" and "customer" in context:
        
            #store the response in true_agent_response
            true_agent_response = text
            df = df.append({"convo_id": convo_id, "context": context.replace('agent: '+text,''), "true_agent_response": true_agent_response}, ignore_index=True) # index=False

    # empty row to separate conversations
    convo_id += 1

#save as csv
df.to_excel("data/evaluate_final_data.xlsx", index=False)


###### Evaluation excel file created, move to evaluation file